## SIMD JULIA

In [19]:
struct VecElement{T}
    value::T
end

In [20]:
const m128 = NTuple{4,VecElement{Float32}}

function add(a::m128, b::m128)
    (VecElement(a[1].value+b[1].value),
     VecElement(a[2].value+b[2].value),
     VecElement(a[3].value+b[3].value),
     VecElement(a[4].value+b[4].value))
end

add (generic function with 1 method)

In [5]:
code_native(add, (m128, m128))

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.globl	_julia_add_1328                 ; -- Begin function julia_add_1328
	.p2align	2
_julia_add_1328:                        ; @julia_add_1328
; ┌ @ In[2]:3 within `add`
	.cfi_startproc
; %bb.0:                                ; %top
; │ @ In[2]:4 within `add`
; │┌ @ float.jl:383 within `+`
	ldr	q0, [x0]
	ldr	q1, [x1]
	fadd	v0.4s, v0.4s, v1.4s
; │└
	str	q0, [x8]
	ret
	.cfi_endproc
; └
                                        ; -- End function
.subsections_via_symbols


In [25]:
v = m128(VecElement.(Float32.((2,2,4,5))))

(VecElement{Float32}(2.0f0), VecElement{Float32}(2.0f0), VecElement{Float32}(4.0f0), VecElement{Float32}(5.0f0))

In [29]:
x = Float32.((2,2,4,5))

(2.0f0, 2.0f0, 4.0f0, 5.0f0)

In [30]:
using BenchmarkTools

In [32]:
@benchmark x.+x

BenchmarkTools.Trial: 10000 samples with 455 evaluations.
 Range (min … max):  231.136 ns …   3.518 μs  ┊ GC (min … max): 0.00% … 90.43%
 Time  (median):     238.187 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   243.758 ns ± 109.795 ns  ┊ GC (mean ± σ):  1.52% ±  3.14%

     ▁█▄▂                                                        
  ▁▂▃████▇▅▃▂▃▃▃▅▅█▇▅▄▄▃▂▂▃▂▂▂▃▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  231 ns           Histogram: frequency by time          267 ns <

 Memory estimate: 128 bytes, allocs estimate: 3.

In [36]:
@benchmark add($v,$v)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.084 ns … 23.459 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.167 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.212 ns ±  0.401 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁    █    ▇    ▄    ▂   ▂    ▁    ▁                    ▁
  ▃▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁▇▁▁▁▁▇▁▁▁▁▆▁▁▁▇ █
  2.08 ns      Histogram: log(frequency) by time     2.58 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [37]:
using SIMD

In [41]:
v_vec = Vec( Float32.((2,2,4,5)))

<4 x Float32>[2.0, 2.0, 4.0, 5.0]

In [43]:
@benchmark v_vec + v_vec

BenchmarkTools.Trial: 10000 samples with 998 evaluations.
 Range (min … max):  18.119 ns …  1.249 μs  ┊ GC (min … max): 0.00% … 95.09%
 Time  (median):     18.662 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.728 ns ± 27.836 ns  ┊ GC (mean ± σ):  3.36% ±  2.35%

  ▅▇▆██▅▂▂▂▁▂▅▄▃▂▁                                            ▂
  █████████████████▇▇▇████▇▇▇█▇█▇▆▆▆▅▄▄▅▆▅▅▄▄▄▄▄▃▅▃▅▄▃▄▃▃▃▁▄▄ █
  18.1 ns      Histogram: log(frequency) by time      26.6 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

In [44]:
@code_llvm add(v,v)

;  @ In[20]:3 within `add`
define void @julia_add_2810([4 x [1 x float]]* noalias nocapture sret([4 x [1 x float]]) %0, [4 x [1 x float]]* nocapture nonnull readonly align 4 dereferenceable(16) %1, [4 x [1 x float]]* nocapture nonnull readonly align 4 dereferenceable(16) %2) #0 {
top:
;  @ In[20]:4 within `add`
; ┌ @ float.jl:383 within `+`
   %3 = bitcast [4 x [1 x float]]* %1 to <4 x float>*
   %4 = load <4 x float>, <4 x float>* %3, align 4
   %5 = bitcast [4 x [1 x float]]* %2 to <4 x float>*
   %6 = load <4 x float>, <4 x float>* %5, align 4
   %7 = fadd <4 x float> %4, %6
; └
  %8 = bitcast [4 x [1 x float]]* %0 to <4 x float>*
  store <4 x float> %7, <4 x float>* %8, align 4
  ret void
}


In [45]:
@code_llvm v_vec + v_vec

;  @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/simdvec.jl:252 within `+`
define void @"julia_+_2812"([1 x <4 x float>]* noalias nocapture sret([1 x <4 x float>]) %0, [1 x <4 x float>]* nocapture nonnull readonly align 16 dereferenceable(16) %1, [1 x <4 x float>]* nocapture nonnull readonly align 16 dereferenceable(16) %2) #0 {
top:
;  @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/simdvec.jl:253 within `+`
; ┌ @ Base.jl:38 within `getproperty`
   %3 = getelementptr inbounds [1 x <4 x float>], [1 x <4 x float>]* %1, i64 0, i64 0
   %4 = getelementptr inbounds [1 x <4 x float>], [1 x <4 x float>]* %2, i64 0, i64 0
; └
; ┌ @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:212 within `fadd` @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:212
; │┌ @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:221 within `macro expansion`
    %5 = load <4 x float>, <4 x float>* %3, align 16
    %6 = load <4 x float>, <4 x float>* %4, align 16